In [1]:
import numpy as np
import sympy as sp
from IPython.display import display, Math

# Class

In [49]:
from abc import ABC, abstractmethod
from typing import Tuple


class Task(ABC):
    @abstractmethod
    def get_result(self):
        pass


class MonopolistTask(Task):
    x1, x2 = sp.symbols('x_1, x_2')
    p1, p2 = sp.symbols('p_1, p_2')

    def __init__(self, x_1, x_2, C):
        self.x_1 = x_1
        self.x_2 = x_2
        self.C = C

    def __do_calculations(self):
        p_1 = self.__express(self.x_1, self.p1, "x_1")[1]
        p_2 = self.__express(self.x_2, self.p2, "x_2")[1]
        p1, p2 = self.p1, self.p2
        x1, x2 = self.x1, self.x2
        R = p1*x1 + p2*x2
        R_simp = R.subs({p1: p_1, p2: p_2})

        P_uneval = R_simp - sp.UnevaluatedExpr(C)
        P = P_uneval.doit()

        res = sp.solve([sp.Eq(0, P.diff(x1)), sp.Eq(0, P.diff(x2))])
        p_1_val = res[x1]
        p_2_val = res[x2]

        x_1_val = x_1.subs({p1: p_1_val})
        x_2_val = x_2.subs({p2: p_2_val})

        E_d1 = (x_1.diff() * p1/x_1).simplify()
        E_d2 = (x_2.diff() * p2/x_2).simplify()
        E_d1_value = E_d1.subs(p1, p_1_val)
        E_d2_value = E_d2.subs(p2, p_2_val)

        coef1 = P.diff(x1).diff(x1)  # глав диагональ
        coef2 = P.diff(x2).diff(x2)  # глав диагональ
        coef3 = P.diff(x1).diff(x2)  # смешанные =>побочная диагональ
        coef4 = P.diff(x2).diff(x1)  # смешанные =>побочная диагональ
        delta1 = coef1  # 1-ый минор
        delta2 = coef1*coef2 - coef3*coef4  # 2-ой минор
        income = R_simp.subs({x1: p_1_val, x2: p_2_val})
        m = np.array([
            [coef1, coef3],
            [coef4, coef2],
        ], dtype=float)

        return p_1, p_2, R, R_simp, P_uneval, P, p_1_val, p_2_val,\
            E_d1, E_d2, E_d1_value, E_d2_value,\
            delta1, delta2, income, m

    def get_result(self):
        result = self.__do_calculations()
        result_out = DisplaySolution(result).display()
        return result_out

    def __express(self, a, b, name: str) -> tuple:
        # выразить одну через другую
        sym = sp.symbols(name)
        sol = sp.solve(a-sym, b)
        assert len(sol) == 1
        return (sym, sol[0])


class DisplaySolution:

    def __init__(self, variables):
        self.variables = variables

    def display(self):
        p_1, p_2, R, R_simp, P_uneval, P, p_1_val, p_2_val,\
            E_d1, E_d2, E_d1_value, E_d2_value,\
            delta1, delta2, income, m = self.variables
        C_symb = sp.symbols("C")
        R_symb = sp.symbols("R")
        P_symb = sp.symbols("P")
        E_d1_symb = sp.symbols("E_d_1")
        E_d2_symb = sp.symbols("E_d_2")
        p1, p2 = sp.symbols('p_1, p_2')

        display(sp.Eq(p_1, p1))
        display(sp.Eq(p_2, p2))
        display(sp.Eq(C_symb, C))
        display(sp.Eq(R_symb, R))
        # display(sp.Eq(R_symb-sp.UnevaluatedExpr(C_symb), P_uneval))
        display(Math(rf"П(x_1,\ x_2)= {sp.latex(P)}"))
        display(Math(f"П(x_1)'= {sp.latex(sp.Eq(P.diff(x1), 0))}"))
        display(Math(f"П(x_2)'= {sp.latex(sp.Eq(P.diff(x2), 0))}"))
        display(Math(f"x_1 = {p_1_val}"))
        display(Math(f"x_2 = {p_2_val}"))
        display(R)
        display(R_simp)
        display(Math(f"Доход:\ {income}"))
        display(sp.Eq(E_d1_symb, E_d1))
        # оно того стоило
        display(Math(rf"E_{{d_1}}(p_1={p_1_val}) = {sp.latex(E_d1_value)}"))
        display(sp.Eq(E_d2_symb, E_d2))
        display(Math(rf"E_{{d_2}}(p_2={p_2_val}) = {sp.latex(E_d2_value)}"))
        display(Math(rf"G = {sp.latex(sp.Matrix(m))}"))
        display(Math(
            r"Чтобы\ убедиться\ в\ том,\ что\ мы\ нашли\ точку\ максимума\ Δ_1 < 0;\ Δ_2 > 0"))
        display(Math(rf"Δ_1 = {delta1};\ Δ_2 = {delta2}"))


# Task text

In [50]:
print("\nЗадача монополист:")
print("Даны функции спроса на каждый продукт:")
display(Math(r'x_1=70-p_1,\ x_2=76-2p_2'))
print("Функция издержек фирмы-монополиста имеет вид")
display(Math(r'C(x_1,x_2) = 4x_1^{2}+5x_1x_2 + x_2^{2}'))
print("Найти производственный план, максимизируюзий прибыль фирмы.\nНайти эластичность спроса на каждый продукт по цене")


Задача монополист:
Даны функции спроса на каждый продукт:


<IPython.core.display.Math object>

Функция издержек фирмы-монополиста имеет вид


<IPython.core.display.Math object>

Найти производственный план, максимизируюзий прибыль фирмы.
Найти эластичность спроса на каждый продукт по цене


# Variables

In [51]:
x1, x2 = sp.symbols('x_1, x_2')
p1, p2 = sp.symbols('p_1, p_2')

x_1 = 70 - p1
x_2 = 76 - 2*p2
C = (4 * x1 ** 2) + (5 * x1 * x2) + (x2 ** 2)

m = MonopolistTask(x_1, x_2, C)

m.get_result()

Eq(70 - x_1, p_1)

Eq(38 - x_2/2, p_2)

Eq(C, 4*x_1**2 + 5*x_1*x_2 + x_2**2)

Eq(R, p_1*x_1 + p_2*x_2)

<IPython.core.display.Math object>

...

Eq(E_d_2, p_2/(p_2 - 38))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>